# get_event_info
This notebook creates event-related node and relationship files for the RADx-KG:

Event.csv
Event-RELATED_TO-CoordinationCenter.csv

In [1]:
import os
import shutil
import pandas as pd
import hashlib
import utils
import publication_query
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [2]:
KG_PATH = "../kg/data"

In [3]:
# Event spreadsheet
doc_id = "1ZPvaKqHYIusiIROpjImlXLC2tqZFnkwb"
grid_id = "1054557813"
presentations = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{doc_id}/export?format=csv&gid={grid_id}", dtype=str, keep_default_na=False)
# assign unique ID to each presentation
presentations["presentationId"] = presentations["eventId"] + presentations["title"]
presentations["presentationId"] = presentations["presentationId"].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
presentations.head()
print(presentations.shape[0])
presentations.head()

87


,eventId,presenters,title,presentationUrl,videoUrl,Subproject,Project,Contact PI(s),PI(s),Affiliation,presentationId
0,2023-11-07_E1,Mark Albers,Longitudinal smell testing to detect SARS-CoV-2 infection,,,Chemosensory testing,U01DC019579-01,Mark Albers,,"ADK Group, LLC",ae27baf31f88698080a13283b977faf3
1,2023-11-07_E1,John Hayes,"The ArOMa-T, a tool for rapid, adaptive testing of olfactory thresholds",,,,,Steven D Munger,,The Pennsylvania State University,962bcba40d8ae53eb9f6adfa92e476de
2,2023-11-07_E1,Christopher Simons,Confectionary-based screening tool to assess chemosensory loss in COVID-19 patients,,,Chemosensory testing,R01DC016112-04S1,Susan Travers,,Ohio State University,78e761fdd29e4c8a1435e1bd0fd35a6e
3,2023-11-07_E1,Pamela Dalton,SCENTinel and other screening tools for remote testing,,,,,Pamela Dalton,,Monell Chemical Senses Center,006458e5ddfa0299c0c5310106b61ce8
4,2023-09-21_E1,Xudong Fan,COVID-19 detection through scent analysis with a compact GC device,,,Scent,U18TR003812-01,Xudong Fan,Kevin Ward,University of Michigan at Ann Arbor,cf197e5a369f99145e525699089439fb


## Create Presentation-PRESENTED_AT-Event relationships

In [4]:
presentations_map = {"presentationId": "from", "eventId": "to"}
presented = utils.rename_and_reorder_columns(presentations, presentations_map)
presented.to_csv(os.path.join(KG_PATH, "relationships", "Presentation-PRESENTED_AT-Event.csv"), index=False)
print(f"Number of Presentation nodes: {presented.shape[0]}")
presented.head()

Number of Presentation nodes: 87


,from,to
0,ae27baf31f88698080a13283b977faf3,2023-11-07_E1
1,962bcba40d8ae53eb9f6adfa92e476de,2023-11-07_E1
2,78e761fdd29e4c8a1435e1bd0fd35a6e,2023-11-07_E1
3,006458e5ddfa0299c0c5310106b61ce8,2023-11-07_E1
4,cf197e5a369f99145e525699089439fb,2023-09-21_E1


In [5]:
#presentations["id"] = presentations["title"].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
#presentations.head()

In [6]:
presenters = presentations[["eventId", "presenters", "presentationId"]].copy()
presenters["presenters"] = presenters["presenters"].str.split("|")
presenters = presenters.explode("presenters")
publication_query.expand_name_column(presenters, "presenters")
presenters["presenters"] = presenters["name"]
presenters.drop(columns="name", inplace=True)
presenters.head()

,eventId,presenters,presentationId,fullName,firstName,middleName,lastName
0,2023-11-07_E1,Albers M,ae27baf31f88698080a13283b977faf3,Mark Albers,Mark,,Albers
1,2023-11-07_E1,Hayes J,962bcba40d8ae53eb9f6adfa92e476de,John Hayes,John,,Hayes
2,2023-11-07_E1,Simons C,78e761fdd29e4c8a1435e1bd0fd35a6e,Christopher Simons,Christopher,,Simons
3,2023-11-07_E1,Dalton P,006458e5ddfa0299c0c5310106b61ce8,Pamela Dalton,Pamela,,Dalton
4,2023-09-21_E1,Fan X,cf197e5a369f99145e525699089439fb,Xudong Fan,Xudong,,Fan


In [7]:
investigators = pd.read_csv(os.path.join(KG_PATH, "nodes", "Researcher_investigators.csv"), dtype=str, keep_default_na=False)
other = pd.read_csv(os.path.join(KG_PATH, "nodes", "Researcher_primary_coauthors.csv"), dtype=str, keep_default_na=False)
researchers = pd.concat([investigators, other])
researchers.head()

,id,name,fullName,firstName,middleName,lastName,orcid,profileId
0,profileid:2563052,Allen CE,Carl E Allen,Carl,E,Allen,orcid:0000-0002-6625-739X,profileid:2563052
1,profileid:7039414,Annapragada AV,Ananth V Annapragada,Ananth,V,Annapragada,orcid:0000-0002-3156-9617,profileid:7039414
2,profileid:10320851,Bassiri H,Hamid Bassiri,Hamid,,Bassiri,orcid:0000-0001-6532-8478,profileid:10320851
3,profileid:7989301,Burns JC,Jane C Burns,Jane,C,Burns,orcid:0000-0001-5679-1217,profileid:7989301
4,profileid:8667619,Chiu CY,Charles Yen Chiu,Charles,Yen,Chiu,orcid:0000-0003-2915-2094,profileid:8667619


In [8]:
presenters_matched = utils.fuzzy_merge(presenters, researchers, left_fuzzy_on="presenters", right_fuzzy_on="name", how="inner", threshold=0.95)
presenters_matched.head()

,eventId,presenters,presentationId,fullName_x,firstName_x,middleName_x,lastName_x,match,score,id,name,fullName_y,firstName_y,middleName_y,lastName_y,orcid,profileId
0,2023-11-07_E1,Albers M,ae27baf31f88698080a13283b977faf3,Mark Albers,Mark,,Albers,Albers MW,0.977778,profileid:6625336,Albers MW,Mark W Albers,Mark,W,Albers,orcid:0000-0001-7855-3455,profileid:6625336
1,2023-11-07_E1,Hayes J,962bcba40d8ae53eb9f6adfa92e476de,John Hayes,John,,Hayes,Hayes JE,0.975000,profileid:9451051,Hayes JE,John Edward Hayes,John,Edward,Hayes,,profileid:9451051
2,2023-11-07_E1,Dalton P,006458e5ddfa0299c0c5310106b61ce8,Pamela Dalton,Pamela,,Dalton,Dalton PH,0.977778,profileid:1877373,Dalton PH,Pamela Helen Dalton,Pamela,Helen,Dalton,orcid:0000-0003-2474-2888,profileid:1877373
3,2023-09-21_E1,Fan X,cf197e5a369f99145e525699089439fb,Xudong Fan,Xudong,,Fan,Fan X,1.000000,profileid:8485696,Fan X,Xudong Fan,Xudong,,Fan,orcid:0000-0003-0149-1326,profileid:8485696
4,2023-09-21_E1,Fu X,41d9821dc51fbf7d26d14dc4ac2a0d54,Xiao-An Fu,Xiao-An,,Fu,Fu X,1.000000,profileid:10807170,Fu X,Xiao-An Fu,Xiao-An,,Fu,,profileid:10807170


## Create Researcher-PRESENTED-Presentation relationships

In [9]:
presented = presenters_matched[["id", "presentationId"]].copy()
presenters_map = {"id": "from", "presentationId": "to"}
presented = utils.rename_and_reorder_columns(presented, presenters_map)
presented.drop_duplicates(inplace=True)
presented = presented[(presented["from"] != "") & (presented["to"] != "")]
presented.to_csv(os.path.join(KG_PATH, "relationships", "Researcher-PRESENTED-Presentation.csv"), index=False)
print(f"Number of other Researcher nodes: {presented.shape[0]}")
presented.head()

Number of other Researcher nodes: 174


,from,to
0,profileid:6625336,ae27baf31f88698080a13283b977faf3
1,profileid:9451051,962bcba40d8ae53eb9f6adfa92e476de
2,profileid:1877373,006458e5ddfa0299c0c5310106b61ce8
3,profileid:8485696,cf197e5a369f99145e525699089439fb
4,profileid:10807170,41d9821dc51fbf7d26d14dc4ac2a0d54


## Create Presentation Nodes

In [10]:
presentations_map = {"presentationId": "id", "title": "name", "presenters": "presenters", "presentationUrl": "presentationUrl",	"videoUrl": "videoUrl"}
presentations = utils.rename_and_reorder_columns(presentations, presentations_map)
presentations.drop_duplicates(inplace=True)
presentations.to_csv(os.path.join(KG_PATH, "nodes", "Presentation.csv"), index=False)
print(f"Number of Presentation nodes: {presentations.shape[0]}")
presentations.head()

Number of Presentation nodes: 87


,id,name,presenters,presentationUrl,videoUrl
0,ae27baf31f88698080a13283b977faf3,Longitudinal smell testing to detect SARS-CoV-2 infection,Mark Albers,,
1,962bcba40d8ae53eb9f6adfa92e476de,"The ArOMa-T, a tool for rapid, adaptive testing of olfactory thresholds",John Hayes,,
2,78e761fdd29e4c8a1435e1bd0fd35a6e,Confectionary-based screening tool to assess chemosensory loss in COVID-19 patients,Christopher Simons,,
3,006458e5ddfa0299c0c5310106b61ce8,SCENTinel and other screening tools for remote testing,Pamela Dalton,,
4,cf197e5a369f99145e525699089439fb,COVID-19 detection through scent analysis with a compact GC device,Xudong Fan,,


## Create Event nodes
Event info is maintained manually in the data directory

In [11]:
doc_id = "1ZPvaKqHYIusiIROpjImlXLC2tqZFnkwb"
grid_id = "77214925"
events = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{doc_id}/export?format=csv&gid={grid_id}", usecols=["id", "name", "eventType", "eventUrl", "startDate", "endDate", "city", "state", "country"], dtype=str, keep_default_na=False)
events.to_csv(os.path.join(KG_PATH, "nodes", "Event.csv"), index=False)
print(f"Number of Event nodes: {events.shape[0]}")
events.head()

Number of Event nodes: 14


,id,name,eventType,eventUrl,startDate,endDate,city,state,country
0,2023-11-07_E1,Towards Universal Chemosensory Testing (TUCT 2023),hybrid conference,https://smellandtastetestingforall.monell.org/,2023-11-05,2023-11-07,Philadelphia,PA,United States of America
1,2023-09-21_E1,RADx-rad 2023 Conference,webinar,https://www.radxrad.org/event/radx-rad-2023-conference-final-conference/,2023-09-21,2023-09-21,,,
2,2023-08-15_E1,"RADx Data Hub: Making Datasets Findable and Accessible Using Digital Object Identifiers (DOIs), Compact URIs (CURIEs), and Other Identifiers",webinar,https://radx-hub.nih.gov/info/events,2023-08-15,2023-08-15,,,
3,2023-06-20_E1,Association of Environmental Engineering and Science Professors (AEESP) National Conference,in-person conference,https://aeesp2023.sites.northeastern.edu/,2023-06-20,2023-06-23,Boston,MA,United States of America
4,2023-05-16_E1,RADx Data Hub: Privacy Protection: De-identifying Human Subject Data,webinar,https://radx-hub.nih.gov/info/events,2023-05-16,2023-05-16,,,


In [12]:
#shutil.copy("../data/Event.csv", os.path.join(KG_PATH, "nodes"))

## Create Event-RELATED_TO-CoordinationCenter.csv

In [13]:
#shutil.copy("../data/Event-RELATED_TO-CoordinationCenter.csv", os.path.join(KG_PATH, "relationships"))